In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import unsloth
from art.config.model import get_model_config, ModelConfig
from art.unsloth.state import ModelState

config = get_model_config(
    "Qwen/Qwen2.5-7B-Instruct", output_dir="./.art/models/test", config=None
)
state = ModelState(config)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-02 22:56:08 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.18: Fast Qwen2 patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.109 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-7b-instruct-unsloth-bnb-4bit with actual GPU utilization = 54.58%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.11 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 32768. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up t

[W402 22:56:18.153769504 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 04-02 22:56:19 weight_utils.py:254] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 04-02 22:56:22 model_runner.py:1115] Loading model weights took 6.7252 GB
INFO 04-02 22:56:22 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-02 22:56:24 worker.py:267] Memory profiling takes 2.20 seconds
INFO 04-02 22:56:24 worker.py:267] the current vLLM instance can use total_gpu_memory (79.11GiB) x gpu_memory_utilization (0.55) = 43.18GiB
INFO 04-02 22:56:24 worker.py:267] model weights take 6.73GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 4.72GiB; the rest of the memory reserved for KV Cache is 31.58GiB.
INFO 04-02 22:56:24 executor_base.py:111] # cuda blocks: 36958, # CPU blocks: 7021
INFO 04-02 22:56:24 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 18.05x
INFO 04-02 22:56:27 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error

Capturing CUDA graph shapes: 100%|██████████| 43/43 [00:26<00:00,  1.62it/s]

INFO 04-02 22:56:53 model_runner.py:1562] Graph capturing finished in 27 secs, took 4.74 GiB
INFO 04-02 22:56:53 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 31.65 seconds



Unsloth 2025.3.18 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
import asyncio
from art.unsloth.train import train

train_task = asyncio.create_task(train(state.trainer, asyncio.Queue()))

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 3 | Total steps = 300,000
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 20,185,088/7,000,000,000 (0.29% trained)


Unsloth: Will smartly offload gradients to save VRAM!


In [16]:
from art.types import TuneConfig
from art.unsloth.service import TuneInputs
import torch

shape = (1, 32768)

state.inputs_queue.put_nowait(
    TuneInputs(
        tokens=torch.randint(0, 100, shape),
        group_ids=torch.randint(0, 10, shape),
        parent_ids=torch.randint(0, 10, shape),
        input_pos=torch.tensor([list(range(shape[1]))]),
        assistant_mask=torch.ones(shape, dtype=torch.bool),
        logprobs=torch.zeros(shape),
        advantages=torch.zeros(shape),
        weights=torch.ones(shape),
        config=TuneConfig(lr=0.0, kl_coef=0.0),
    )
)

In [ ]:
from accelerate.optimizer import AcceleratedOptimizer

In [50]:
from accelerate.optimizer import AcceleratedOptimizer

if isinstance(state.trainer.optimizer, AcceleratedOptimizer):
    import torch

    def move_tensors(obj, to):
        if isinstance(obj, torch.Tensor):
            return obj.to(to)
        elif isinstance(obj, dict):
            return {key: move_tensors(value, to) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [move_tensors(item, to) for item in obj]
        elif isinstance(obj, tuple):
            return tuple(move_tensors(item, to) for item in obj)
        else:
            return obj

    state.trainer.optimizer.optimizer.state_dict()["state"] = move_tensors(state.trainer.optimizer.optimizer.state_dict()["state"], torch.device("cuda:0"))

In [13]:
from art.unsloth.train import free_memory

free_memory()

In [31]:
state.trainer.optimizer.optimizer.state_dict()["param_groups"]

[{'weight_decay': 0.1,
  'lr': 5e-06,
  'betas': (0.9, 0.99),
  'eps': 1e-08,
  'alpha': 0.0,
  't_alpha': None,
  't_beta3': None,
  'initial_lr': 5e-06,
  'params': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
 

In [43]:
train_task.result()

InvalidStateError: Result is not ready.